In [1]:
from pynwb import NWBFile, NWBHDF5IO, load_namespaces, register_class, TimeSeries
from pynwb.spec import NWBGroupSpec, NWBDatasetSpec, NWBAttributeSpec, NWBNamespaceBuilder, SpecNamespace
from pynwb.file import MultiContainerInterface, NWBContainer
from hdmf.utils import docval, get_docval, popargs

from datetime import datetime
from dateutil.tz import tzlocal
import numpy as np

In [2]:
SpO2 = NWBGroupSpec(
    doc='Stores blood oxygen saturation levels',
    attributes=[
        NWBAttributeSpec(
            name='device',
            dtype='text',
            doc='Identifier for the device type'
        )
    ],
    # datasets=[
    #     NWBDatasetSpec(
    #         name='SpO2',
    #         doc='Blood oxygen saturation levels',
    #         dtype='float'
    #     )
    # ],
    neurodata_type_def='SpO2',
    neurodata_type_inc='TimeSeries'
)

name = "BloodOxygen"
ns_path = name + ".namespace.yaml"
ext_source = name + ".extensions.yaml"

ns_builder = NWBNamespaceBuilder(name + " extensions", name, version="0.1.0")
ns_builder.include_type("TimeSeries", namespace="core")
ns_builder.add_spec(ext_source, SpO2)
ns_builder.export(ns_path)

In [3]:
load_namespaces(ns_path)

@register_class("SpO2", name)
class SpO2(TimeSeries):
    __nwbfields__ = ('device',)

    @docval(
        *get_docval(TimeSeries.__init__) #name, data, timestamps, description, units
        + ( #{'name':'SpO2', 'type':('array_data', 'data'), 'doc':'Blood oxygen saturation level'},
            {'name':'device', 'type':str, 'doc':'Identifier for the device type'},
          )
    )

    def __init__(self, **kwargs):
        # SpO2 = popargs('SpO2', kwargs)
        device = popargs('device', kwargs)
        super().__init__(**kwargs)
        # self.SpO2 = SpO2
        self.device = device

In [6]:
nwbfile = NWBFile(
    session_description='Session with blood oxygen data',
    identifier='id1',
    session_start_time=datetime.now(tz=tzlocal())
)

example_data = [98.0, 96.5, 97.2]
blood_oxygen = SpO2(
    name='example_spo2',
    # data=np.array([], dtype=float),
    # SpO2=example_data,
    data=example_data,
    timestamps=[1.0, 2.0, 3.0],
    device='Apple Watch Series 6',
    description='Blood oxygen saturation levels collected from an Apple Watch',
    unit='%'
)

nwbfile.add_acquisition(blood_oxygen)

with NWBHDF5IO('blood_oxygen_data.nwb', 'w') as io:
    io.write(nwbfile)

In [16]:
with NWBHDF5IO('blood_oxygen_data.nwb', 'r') as io:
    nwbfile = io.read()
    blood_oxygen_data = nwbfile.acquisition['example_spo2']
    print(f'SpO2 Data: {blood_oxygen_data.data[:]}')
    print(f'Device: {blood_oxygen_data.device}')

SpO2 Data: [98.  96.5 97.2]
Device: Apple Watch Series 6
